In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.types import StructType,StructField, StringType, IntegerType , BooleanType
spark = SparkSession.builder.appName('taxi').getOrCreate()
sc = spark.sparkContext

In [7]:
df = spark.read.options(header='true', inferschema='true').csv("nyc_taxi.csv")
df.printSchema()
df.show(5,False)

root
 |-- timestamp: string (nullable = true)
 |-- value: integer (nullable = true)

+-------------------+-----+
|timestamp          |value|
+-------------------+-----+
|2014-07-01 00:00:00|10844|
|2014-07-01 00:30:00|8127 |
|2014-07-01 01:00:00|6210 |
|2014-07-01 01:30:00|4656 |
|2014-07-01 02:00:00|3820 |
+-------------------+-----+
only showing top 5 rows



In [22]:
# Add predefined quantiles with rolling updates in the future
THRESH = 1.075

Q1, Q3 = df.approxQuantile("value", [0.25,0.75],0.0)
IQR = Q3 - Q1
lower_thres = Q1 - THRESH*IQR
upper_thres = Q3+ THRESH*IQR

outliers = df.filter((f.col('value') < lower_thres) | (f.col('value') > upper_thres))
outliers.show()

10259.0 19838.0
+-------------------+-----+
|          timestamp|value|
+-------------------+-----+
|2014-09-06 22:30:00|30313|
|2014-09-06 23:00:00|30373|
|2014-11-02 01:00:00|39197|
|2014-11-02 01:30:00|35212|
|2015-01-01 01:00:00|30236|
+-------------------+-----+

